<a href="https://colab.research.google.com/github/Um-e-Hani/DevOps-R-Repo1111/blob/main/Code/LoadedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#convert output category value to tag
def logits_to_tokens(output, index, test_sent):
  token_sequences = []
  token_num = len(test_sent)
  count = 0
  for categorical_sequence in output:
    token_sequence = []
    for categorical in categorical_sequence:
      token_sequence.append(tagList[np.argmax(categorical)-1])
      count = count + 1
      if(token_num == count):
        break
    token_sequences.append(token_sequence)
  return token_sequences

#function to prepare a sentence to allow model POS prediction
def getSentence_x(sentence, word2index):
  sentence_index = []
  for word in sentence:
      try:
          sentence_index.append(word2index[word])
      except KeyError:
          sentence_index.append(word2index['-OOV-'])
  return sentence_index

#predict function output
def predictFunct(test_sentence, sentOriginal):
  l1 = []
  l1.append(test_sentence.tolist())
  output = model.predict(l1)

  #converting output to readable form
  p_tags = logits_to_tokens(output, tagList, sentOriginal)
  return p_tags

In [2]:
# load json module
import json

#reading word2Index
with open("/content/drive/MyDrive/FYP/POS-Tagging-Urdu/files/word2index") as f:
  word2index = json.loads(f.read())

#reading tagList
with open("/content/drive/MyDrive/FYP/POS-Tagging-Urdu/files/tagList") as fp:
  tagList = json.loads(fp.read())


In [3]:
#Keras

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Masking, TimeDistributed, LSTM, Conv1D, Embedding
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Reshape, Dropout, Dense,Multiply, Dot, Concatenate,Embedding
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

#reconstructing model
model = keras.models.load_model("/content/drive/MyDrive/FYP/POS-Tagging-Urdu/models/1/mlp.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 281, 128)          5332736   
                                                                 
 dense (Dense)               (None, 281, 128)          16512     
                                                                 
 dense_1 (Dense)             (None, 281, 41)           5289      
                                                                 
 activation (Activation)     (None, 281, 41)           0         
                                                                 
Total params: 5,354,537
Trainable params: 5,354,537
Non-trainable params: 0
_________________________________________________________________


In [9]:
!pip install flask-ngrok
!pip install pydub
!pip install SpeechRecognition
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio
!pip install gevent
!pip install ngrok

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 5.8 MB 21.8 MB/s 
     |████████████████████████████████| 251 kB 55.3 MB/s 


In [19]:
#python flask application
from flask import Flask,request,jsonify
from flask_ngrok  import run_with_ngrok

#import library
import speech_recognition as sr
import numpy as np

# Initialize recognizer class (for recognizing the speech)
r = sr.Recognizer()

app = Flask(__name__)

@app.route('/upload', methods=['GET', 'POST'])
def hello_world():
  if request.method == 'POST':
    f = request.files['file']
    #sound = AudioSegment.from_mp3(f)
    #sound.export(f, format="wav")
    filePath = "./audio.wav"
    f.save(filePath)
    
    # Reading Audio file as source
    # listening the audio file and store in audio_text variable
    with sr.AudioFile('audio.wav') as source:
      audio_text = r.listen(source)
        
    # recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
    try:   
        # using google speech recognition
        sentence = r.recognize_google(audio_text, language = "ur-PK")
        print('Converting audio transcripts into text ...')
        print(sentence)
    except:
         print('Sorry.. run again...')

    #getting tags for sentence
    sentOriginal = np.array(sentence.split(" "))
    sent = np.array(getSentence_x(sentOriginal, word2index))

    #converting output to readable form
    val = 281 - int(sent.shape[0])
    new = np.pad(sent, (0, val), 'constant')
    p_tags = predictFunct(new, sentOriginal)

    #creating return string
    retStr = ""
    for x in range(0,len(p_tags[0])):
      retStr = retStr + sentOriginal[x] + "|" + p_tags[0][x] + " "
    print(retStr)

    return jsonify({
		"message": retStr
	  })

run_with_ngrok(app)
#if __name__ == "__main__":
app.run()

Converting audio transcripts into text ...
وہ دسویں جماعت میں پڑھتا ہے
['وہ' 'دسویں' 'جماعت' 'میں' 'پڑھتا' 'ہے']
[['PP', 'OR', 'NN', 'P', 'VB', 'TA']]
وہ|PP دسویں|OR جماعت|NN میں|P پڑھتا|VB ہے|TA 
